In [1]:
!pip install transformers
!pip install datasets


In [2]:
!pip install evaluate


In [3]:
# !pip install accelerate -U
# !pip install accelerate==0.20.1

In [4]:
from datasets import load_dataset
#dataset = load_dataset('GonzaloA/fake_news')
dataset = load_dataset('amazon_polarity')



In [5]:
# 列出数据集对象中的列名
print(dataset.column_names)


{'train': ['label', 'title', 'content'], 'test': ['label', 'title', 'content']}


In [6]:
# 假设数据集对象为 dataset
labels_train = dataset['train']['label']
labels_test = dataset['test']['label']

# 查看部分训练集和测试集的标签内容
print(labels_train[:5])  # 前五个训练集标签
print(labels_test[:5])   # 前五个测试集标签


[1, 1, 1, 1, 1]
[1, 1, 0, 1, 1]


In [7]:
# model_checkpoint = 'bert-base-uncased'
#

In [8]:
# # Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")
# model.classifier = new_classifier

In [9]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


In [10]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)


In [11]:
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)

In [12]:
# tokenized_datasets_news = dataset.map(tokenize_function, batched=True)
# tokenized_datasets_news.save_to_disk("/Users/liang/Downloads/tokenized_datasetsnews")
from datasets import load_from_disk

# 加载tokenized数据集
tokenized_datasets_news = load_from_disk("/Users/liang/Downloads/tokenized_datasetsnews")


In [13]:
small_train_dataset = tokenized_datasets_news["train"].shuffle(seed=42).select(range(1500))
small_eval_dataset = tokenized_datasets_news["test"].shuffle(seed=42).select(range(1500))


In [14]:
# 可選：凍結模型層（如果需要）
# for param in model.base_model.parameters():
#     param.requires_grad = False

In [15]:
from transformers import BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# 取得RoBERTa模型的不同層或部分（例如：roberta層）

# for param in model.bert.parameters():
#     param.requires_grad = False
roberta_layers = model.roberta
for param in roberta_layers.parameters():
    param.requires_grad = False

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


In [17]:
# from sklearn.metrics import accuracy_score, f1_score
# import numpy as np

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# def compute_metrics(predictions, references):
#     # Convert the references to a flat list
#     flat_references = [label for sublist in references for label in sublist]

#     # Calculate the predictions
#     flat_predictions = [pred.argmax() for sublist in predictions for pred in sublist]

#     # Calculate the metrics
#     accuracy = accuracy_score(flat_references, flat_predictions)
#     precision = precision_score(flat_references, flat_predictions)
#     recall = recall_score(flat_references, flat_predictions)
#     f1 = f1_score(flat_references, flat_predictions)

#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1
#     }


In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [19]:
!pip install accelerate

In [20]:
# !pip install accelerate==0.20.1 -U
# !pip install transformers[torch]
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=10)


In [21]:
# !pip show transformers
# !pip show accelerate


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)


In [23]:
# !pip install accelerate==0.20.1
# !pip install transformers --upgrade


In [24]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.680207,0.613333
2,No log,0.688469,0.568000
3,0.745200,0.666633,0.602667
4,0.745200,0.647194,0.627333
5,0.745200,0.651685,0.647333
6,0.661500,0.648114,0.598667
7,0.661500,0.633099,0.663333
8,0.645300,0.637065,0.642000
9,0.645300,0.626224,0.678667
10,0.645300,0.627063,0.670667


TrainOutput(global_step=1880, training_loss=0.6744296297113946, metrics={'train_runtime': 1012.1537, 'train_samples_per_second': 14.82, 'train_steps_per_second': 1.857, 'total_flos': 3946665830400000.0, 'train_loss': 0.6744296297113946, 'epoch': 10.0})

In [25]:
model.eval()
evaluation_results = trainer.evaluate()
print(evaluation_results)


{'eval_loss': 0.6270634531974792, 'eval_accuracy': 0.6706666666666666, 'eval_runtime': 46.8963, 'eval_samples_per_second': 31.985, 'eval_steps_per_second': 4.009, 'epoch': 10.0}


In [26]:
test = '''
WNDR shall not be responsible for any incorrect or inaccurate information, whether caused by website users or by any of the equipment or programming associated with or utilized in this website or by any technical or human error which may occur.  WNDR assumes however all responsibility for the satirical nature of its articles and for the fictional nature of their content. All characters appearing in the articles in this website – even those based on real people –  are entirely fictional and any resemblance between them and any persons, living, dead, or undead is purely a miracle.
'''


In [27]:
test1='''this product is very bad'''
test2='''this product is good'''

In [28]:
import torch

inputs = tokenizer(test2, return_tensors="pt")
inputs = {key: value.to(model.device) for key, value in inputs.items()}
model.to(inputs["input_ids"].device)
outputs = model(**inputs)
logits = outputs.logits
predicted_class = logits.argmax(dim=1).item()

print(f"Predicted Class: {predicted_class}")


Predicted Class: 1


In [29]:
real_test = '''
Beijing
CNN
 —
Russian President Vladimir Putin and Chinese leader Xi Jinping met in Beijing Tuesday at the start of a visit expected to underscore the two leaders’ shared vision for a new international order no longer dominated by the United States and its democratic allies.

This meeting between the world’s two most powerful autocrats comes as geopolitical fault lines harden globally – first following Russia’s grinding invasion of Ukraine, launched just three weeks after the leaders’ last meeting in the Chinese capital in 2022 – and now as Israel’s war against Palestinian militant group Hamas threatens to spiral into a broader conflict that could shatter stability in the Middle East.

Both Beijing and Moscow have criticized Israel’s actions and called for a ceasefire, in the latest showing of the two powers’ efforts to step up their alternative leadership to that of the US, which affirms Israel’s on-going right to retaliate.

The meeting between the two men comes as it was confirmed US President Joe Biden would travel to Israel on Wednesday. Biden is expected to give a high-profile show of support for Israel as it attempts to eliminate Hamas, while the US presses for ways to ease humanitarian suffering in Gaza and warns other regional players about becoming further engaged in the conflict.

Xi and Putin are expected to discuss the situation in Gaza during their meeting this week after Putin arrived Tuesday morning as a guest of honor at the Belt and Road Forum in Beijing – an exceptionally rare foreign trip for the diplomatically isolated leader.

Heads of state, representatives and delegations from more than 140 countries are expected to attend China’s landmark, two-day diplomatic event beginning Tuesday, which marks 10 years since the start of Xi’s global infrastructure funding drive, and presents the Chinese leader with an opportunity to project Beijing’s growing global ambitions.'''

In [30]:
inputs = tokenizer(real_test, return_tensors="pt")
inputs = {key: value.to(model.device) for key, value in inputs.items()}
model.to(inputs["input_ids"].device)
outputs = model(**inputs)
logits = outputs.logits
predicted_class = logits.argmax(dim=1).item()

print(f"Predicted Class: {predicted_class}")


Predicted Class: 1
